In [26]:
import os
import shutil
import hickle
import re

import pandas as pd
import tempfile
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

import numpy as np
import PIL
from PIL import Image, ImageChops, ImageDraw
import torch
from torch import nn
import torchvision.models
import cv2

from pathlib import Path

import tensorflow as tf
from tensorflow import keras
import matplotlib.patches as patches

from tqdm import tqdm
from numba import jit, cuda
from utils import systemic_brightening
import warnings

In [ ]:
# set directory
os.chdir("/users/riya/race/classifier_experiments") # which one? yep

# ignore warnings
warnings.filterwarnings("ignore")

# import model
segmentation_classifier = keras.models.load_model('models/MIMIC-256x25680-20-split-resnet-Float16_2-race_detection_rop_seg_data_rop_seg-0.001_20220321-054140_epoch:011.hdf5')

In [17]:
img_arr = np.zeros((4546, 256, 256, 3))

In [19]:
img_arr[0].shape

(256, 256, 3)

In [23]:
# run through classifier

@jit         
def predict_on_images(img_path, preds_df, colname,
                     skeleton, thresh_type, intensity_change, brighten_sum,
                     csv_name = "brightened_predictions"): 
    
    img_files = os.listdir(dataset_path)
    num_images = len(img_files) - 4500
    
    id_arr = [0] * num_images
    img_arr = np.zeros((num_images, 256, 256, 3))
    
    preds_arr = [0] * num_images
    
    for i in tqdm(range(num_images)):
        arr = np.array(Image.open(img_path + img_files[i]))
        resized = cv2.resize(arr, (256,256)) # must use 256
        channels = np.repeat(resized[:, :, np.newaxis], 3, axis=2).reshape((256,256,3))

        modified_img = systemic_brightening(channels, skeleton, thresh_type, intensity_change, brighten_sum,
                                           image_size = (256, 256))
        modified_img = np.array(modified_img) # .reshape((1,256,256,3)) np reshape, bc substitute? 
        # img_arr = np.append(img_arr, modified_img)
        img_arr[i] = modified_img
        
        # print(img_arr.shape)
                       
        # getting id     
        img_id = re.findall(r'\d+', img_files[i])
        id_arr[i] = img_id
    
    preds_df['id'] = id_arr
  
    # getting prediction    
    prediction = segmentation_classifier(img_arr)
    
    for j in range(num_images):
        preds_arr[j] = prediction.numpy()[j, 1] # returning the white prediction for each image
  
    preds_df[colname] = preds_arr           
    preds_df.to_csv(preds_path + csv_name + ".csv")

In [24]:
all_predictions = pd.DataFrame(columns = ['id', '30', '60', '90', '120', '150']) # from id I can get race

dataset_path = "/users/riya/race/dataset/segmentations/"
preds_path = "/users/riya/race/classifier_experiments/predictions/experiment1_plus_systemic_brightening/"

In [25]:
predict_on_images(dataset_path, all_predictions, '30', False, 'below', 'brighten', 30)

<ipython-input-23-add7a222728a>:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "predict_on_images" failed type inference due to: Untyped global name 'tqdm': Cannot determine Numba type of <class 'type'>

File "<ipython-input-23-add7a222728a>", line 16:
def predict_on_images(img_path, preds_df, colname,
    <source elided>
    
    for i in tqdm(range(num_images)):
    ^

  @jit
<ipython-input-23-add7a222728a>:3: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "predict_on_images" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-23-add7a222728a>", line 16:
def predict_on_images(img_path, preds_df, colname,
    <source elided>
    
    for i in tqdm(range(num_images)):
    ^

  @jit
/usr/local/lib/python3.6/dist-packages/numba/core/object_mode_passes.py:152: NumbaWarning: Function "predict_on_images" was

KeyboardInterrupt: 